In [1]:
import pandas as pd
import gensim
import pyLDAvis.gensim

In [2]:
data = pd.read_pickle('../../data/news/preprocessed_data.pkl')

In [3]:
len(data)

96074

# Dictionary encoding

In [4]:
keyword_data = data.키워드.values[:] #위치, 키워드 가능하나 일단은 '위치'

In [5]:
dictionary = gensim.corpora.Dictionary(keyword_data)
corpus = [dictionary.doc2bow(cnt) for cnt in keyword_data]

# Topic model (LDA)

In [6]:
topic_number = 8
number_of_passes = 15  # 기본값 1
number_of_iterations = 50  # 기본값 50
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=topic_number, id2word=dictionary,
                                            passes=number_of_passes,  iterations=number_of_iterations)

In [7]:
lda_model.print_topics()

[(0,
  '0.028*"아파트" + 0.021*"가격" + 0.020*"서울" + 0.019*"부동산" + 0.016*"시장" + 0.016*"지역" + 0.014*"상승" + 0.013*"대책" + 0.012*"분양" + 0.010*"정부"'),
 (1,
  '0.046*"공시" + 0.032*"규모" + 0.021*"계약" + 0.014*"결정" + 0.011*"체결" + 0.010*"회사" + 0.009*"공급" + 0.008*"대비" + 0.007*"변경" + 0.007*"주식"'),
 (2,
  '0.017*"상승" + 0.015*"공시" + 0.015*"거래" + 0.012*"하락" + 0.011*"종목" + 0.011*"지가" + 0.011*"주가" + 0.010*"시장" + 0.010*"개별" + 0.009*"토지"'),
 (3,
  '0.047*"주택" + 0.019*"임대" + 0.013*"공급" + 0.013*"적용" + 0.012*"정부" + 0.011*"가구" + 0.007*"양도" + 0.007*"주거" + 0.007*"지원" + 0.007*"청약"'),
 (4,
  '0.052*"대출" + 0.014*"가계" + 0.014*"은행" + 0.011*"금융" + 0.011*"금리" + 0.010*"규제" + 0.010*"LTV" + 0.010*"부채" + 0.010*"증가" + 0.009*"DTI"'),
 (5,
  '0.028*"사업" + 0.009*"계획" + 0.009*"조합" + 0.009*"공사" + 0.009*"건설" + 0.008*"추진" + 0.007*"서울시" + 0.007*"선정" + 0.007*"서울" + 0.007*"단지"'),
 (6,
  '0.017*"분양" + 0.014*"아파트" + 0.012*"예정" + 0.012*"단지" + 0.008*"전용" + 0.008*"가구" + 0.007*"위치" + 0.007*"오피스텔" + 0.007*"도시" + 0.007*"인근"'),
 (7,
  '0.014*"부동산"

# 시각화

In [9]:
pyLDAvis.enable_notebook()
plot_lda_model = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(plot_lda_model)

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Topics in Docs

In [10]:
show_top_n = 10

print('  Topic Rates')
for cnt, topic_rate in enumerate(lda_model[corpus]):
    if cnt == show_top_n:
        break
    print('{0:>3}th article  :  {1}'.format(cnt, topic_rate))

  Topic Rates
  0th article  :  [(5, 0.5749738), (6, 0.33244708), (7, 0.08951776)]
  1th article  :  [(0, 0.9938982)]
  2th article  :  [(0, 0.21839139), (1, 0.04837707), (3, 0.08691769), (5, 0.13862224), (7, 0.5046739)]
  3th article  :  [(1, 0.047402434), (3, 0.065940194), (4, 0.033318724), (5, 0.64812565), (6, 0.15409416), (7, 0.04968616)]
  4th article  :  [(0, 0.86118984), (3, 0.014372486), (5, 0.036405914), (6, 0.08650204)]
  5th article  :  [(0, 0.96204567), (3, 0.022029694), (4, 0.011435388)]
  6th article  :  [(6, 0.17218237), (7, 0.81663847)]
  7th article  :  [(1, 0.09012741), (3, 0.39464223), (4, 0.14788857), (5, 0.02862838), (7, 0.3369583)]
  8th article  :  [(0, 0.17693985), (1, 0.08891325), (2, 0.55509746), (7, 0.17679714)]
  9th article  :  [(0, 0.5516414), (3, 0.06408343), (5, 0.12457747), (6, 0.25694013)]


C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
def make_topic_data(model, corpus):
    '''
    문서별 최상의 topic을 칼럼으로 가지는 pandas.DataFrame 반환
    model : gensim.models.ldamodel.LdaModel
    corpus : corpus
    '''
    topic_data = pd.DataFrame()

    for cnt, topic_rate_list in enumerate(model[corpus]):
        sort_topic_rate_list = sorted(
            topic_rate_list, key=lambda x: x[1], reverse=True)
        (top_topic_numb, top_topic_rate) = sort_topic_rate_list[0]
        
        topic_data = topic_data.append(pd.Series(
            [int(top_topic_numb), top_topic_rate, sort_topic_rate_list]), ignore_index=True)

    topic_data.columns = ['name_top_topic', 'rate_top_topic', 'topic_detil']
    return topic_data

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
topic_data = make_topic_data(lda_model, corpus)

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
topic_data[:10]

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,name_top_topic,rate_top_topic,topic_detil
0,5.0,0.574979,"[(5, 0.57497853), (6, 0.33244535), (7, 0.08951..."
1,0.0,0.993898,"[(0, 0.99389803)]"
2,7.0,0.504668,"[(7, 0.5046685), (0, 0.21838437), (5, 0.138636..."
3,5.0,0.648130,"[(5, 0.6481301), (6, 0.15409544), (3, 0.065944..."
4,0.0,0.861192,"[(0, 0.8611924), (6, 0.08650907), (5, 0.036408..."
5,0.0,0.962033,"[(0, 0.9620329), (3, 0.022042692), (4, 0.01143..."
6,7.0,0.816638,"[(7, 0.816638), (6, 0.1721823)]"
7,3.0,0.394642,"[(3, 0.39464226), (7, 0.3369586), (4, 0.147888..."
8,2.0,0.555098,"[(2, 0.55509806), (0, 0.17693625), (7, 0.17680..."
9,0.0,0.551638,"[(0, 0.55163807), (6, 0.2569542), (5, 0.124565..."


In [14]:
data.iloc[:10,].reset_index()

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,뉴스 식별자,일자,언론사,제목,위치,키워드,특성추출(가중치순 상위 50개),URL
0,1.500301e+06,2017-01-31,경상일보,"[현장&이슈]천상 주택가 한가운데 주상복합,주민 반발","[울산, 범서읍, 백문현, 천상, 울주군]","[천상, 주택가, 주상, 복합, 주민, 반발, 울산, 울주군, 범서읍, 천상, 주택...","[주민들, 주상복합, 시행사, 울주군, 탑마트, 주택가, 일조권, 신한다솜아파트, ...",www.ksilbo.co.kr/news/articleView.html?idxno=5...
1,1.500301e+06,2017-01-31,경상일보,울산 주택 매매 전세 월세 ‘트리플 약세’,"[울산, 충북, 대전, 전북, 경기, 부산, 제주, 대구, 광주, 경남]","[매매, 울산, 주택, 전세, 월세, 트리플, 약세, 새해, 울산, 주택, 매매, ...","[울산, 월세가격, 전세가격, 3개월, 하락세, 대구, 경남, 충북, 7만, 내림세...",www.ksilbo.co.kr/news/articleView.html?idxno=5...
2,1.500301e+06,2017-01-31,경상일보,"경남지역 아파트, 횡령 부당입찰 비리 여전","[회계, 창원지역, 경남지역, 양산지역, 경남도]","[경남, 지역, 아파트, 횡령, 부당입찰, 비리, 여전, 경남지역, 아파트, 단지,...","[가입자, 관리비, 경남도, 양산, 교체공사, 과태료, 사업자, 10건, 회계담당자...",www.ksilbo.co.kr/news/articleView.html?idxno=5...
3,1.500301e+06,2017-01-31,경상일보,울산 북구 효문공단 잔여지 40여년만에 개발 재개,"[울산, 울산미포국가산업단지, 북구, 울산시, 경기]","[40여년, 울산, 북구, 효문, 공단, 잔여, 재개, 40년, 장기간, 방치, 울...","[효문공단, lh, 토지보상, 그동안, 울산시, 관계자, 장기간, 울산, 잔여지, ...",www.ksilbo.co.kr/news/articleView.html?idxno=5...
4,1.101101e+06,2017-01-31,한국일보,11ㆍ3대책에 얼었던 강남4구 아파트 다시 꿈틀,"[서울, 강남권, 강남구, 구로, 강남, 강남 3구, 신반포, 개포동, 비강남권, ...","[11ㆍ3대책, 강남, 아파트, 강남, 대책, 하락, 12주, 반등, 강남권, 11...","[강남, 강남권, 서울, 분양가, 신반포, 강남구, 잠원동, 부동산, 하락세, 12...",http://hankookilbo.com/v/f8021f7019024e87b693d...
5,1.101101e+06,2017-01-31,한국일보,강남 3구 재건축 아파트 2년새 3억원 급등,"[미국, 동대문구, 서울, 용산구, 강남구, 중랑구, 강남, 노원구, 강남 3구, ...","[2년, 강남, 아파트, 3억, 급등, 서울, 아파트, 가격, 상승세, 강남, 강남...","[3억, 강남, 서울, 부동산, 재건축, 송파구, 상승세, 10억, 1억, 858만...",http://hankookilbo.com/v/6a43ced7df33481cb8341...
6,1.101101e+06,2017-01-31,한국일보,"기아차, ‘스토닉’ 가격 및 사양 ‘프라이드’로 보여","[개성, 미국, 하단, 영국, 유럽]","[기아차, 스토닉, 가격, 사양, 프라이드, 기아자동차, 국내시장, 소형, 스포츠유...","[기아차, 리터, 스토닉, 영국, 가솔린, 특허청, 모터쇼, 축거, 하단부, 1만,...",http://hankookilbo.com/v/665c325912a74317956bc...
7,1.101101e+06,2017-01-31,한국일보,4월부터 장기 저축성보험 비과세 혜택 확 줄어든다,"[아시아, 신성, 경기]","[4월, 장기, 저축, 보험, 혜택, 일시납, 1억, 축소, 적립식, 제한, 중고,...","[사업비, 적립식, 보험료, 일시납, 다이오드, 판매업, 그간, 개정안, 가입분, ...",http://hankookilbo.com/v/e7d6dda333604b1ea08dc...
8,1.101101e+06,2017-01-31,한국일보,‘200만원 터치’ 삼성전자 액면분할 재점화,[None],"[200만, 터치, 재점화, 삼성전자, 액면, 분할, 개인, 1주, 가격, 솔솔, ...","[액면분할, 삼성전자, 황제주, 200만, 투자자, 가능성, 관계자, 원선, 이창목...",http://hankookilbo.com/v/83f9efbc8b044ec6b91f2...
9,1.101101e+06,2017-01-31,한국일보,서울 강남에 아파트 1만8천가구 쏟아진다 11년만에 최대,"[서초동, 서초우성1단지, 서울, 청담동, 강남구, 개포시영, 개포, 주공8단지, ...","[서울, 강남, 아파트, 가구, 11년, 최대, 서울, 강남, 서초, 송파, 강동구...","[강남, 1만, 강남구, 서울, 부동산, 일반분양, 고덕, 강동구, 잠원동, 강남권...",http://hankookilbo.com/v/53e139a28e2340d58208f...


# join with original data

In [15]:
merge_data = pd.concat([data.iloc[:, ].reset_index(), topic_data], axis=1)[['뉴스 식별자', '일자', '언론사', '제목', '위치', 'URL',
                                                                                'name_top_topic', 'rate_top_topic', 'topic_detil']]

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
# key 추가
merge_data['year'] = merge_data.일자.dt.year
merge_data['month'] = merge_data.일자.dt.month
merge_data['day'] = merge_data.일자.dt.day
merge_data['week'] = merge_data.일자.dt.weekofyear

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


In [17]:
merge_data.head(6)

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,뉴스 식별자,일자,언론사,제목,위치,URL,name_top_topic,rate_top_topic,topic_detil,year,month,day,week
0,1.500301e+06,2017-01-31,경상일보,"[현장&이슈]천상 주택가 한가운데 주상복합,주민 반발","[울산, 범서읍, 백문현, 천상, 울주군]",www.ksilbo.co.kr/news/articleView.html?idxno=5...,5.0,0.574979,"[(5, 0.57497853), (6, 0.33244535), (7, 0.08951...",2017,1,31,5
1,1.500301e+06,2017-01-31,경상일보,울산 주택 매매 전세 월세 ‘트리플 약세’,"[울산, 충북, 대전, 전북, 경기, 부산, 제주, 대구, 광주, 경남]",www.ksilbo.co.kr/news/articleView.html?idxno=5...,0.0,0.993898,"[(0, 0.99389803)]",2017,1,31,5
2,1.500301e+06,2017-01-31,경상일보,"경남지역 아파트, 횡령 부당입찰 비리 여전","[회계, 창원지역, 경남지역, 양산지역, 경남도]",www.ksilbo.co.kr/news/articleView.html?idxno=5...,7.0,0.504668,"[(7, 0.5046685), (0, 0.21838437), (5, 0.138636...",2017,1,31,5
3,1.500301e+06,2017-01-31,경상일보,울산 북구 효문공단 잔여지 40여년만에 개발 재개,"[울산, 울산미포국가산업단지, 북구, 울산시, 경기]",www.ksilbo.co.kr/news/articleView.html?idxno=5...,5.0,0.648130,"[(5, 0.6481301), (6, 0.15409544), (3, 0.065944...",2017,1,31,5
4,1.101101e+06,2017-01-31,한국일보,11ㆍ3대책에 얼었던 강남4구 아파트 다시 꿈틀,"[서울, 강남권, 강남구, 구로, 강남, 강남 3구, 신반포, 개포동, 비강남권, ...",http://hankookilbo.com/v/f8021f7019024e87b693d...,0.0,0.861192,"[(0, 0.8611924), (6, 0.08650907), (5, 0.036408...",2017,1,31,5
5,1.101101e+06,2017-01-31,한국일보,강남 3구 재건축 아파트 2년새 3억원 급등,"[미국, 동대문구, 서울, 용산구, 강남구, 중랑구, 강남, 노원구, 강남 3구, ...",http://hankookilbo.com/v/6a43ced7df33481cb8341...,0.0,0.962033,"[(0, 0.9620329), (3, 0.022042692), (4, 0.01143...",2017,1,31,5


In [18]:
merge_data.to_pickle('../../data/news/topic_data.pkl')
merge_data.to_csv('../../data/news/topic_data.csv')

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
